In [1]:
import polars

from autofeat.dataset import *
from autofeat.transform import *

In [2]:
dataset = KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance")

In [3]:
transform = (
    Cast()
    .then(Encode())
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [4]:
filters = [
    Filter(eq={"EmployeeID": "51A4-EA6E"}),
    Filter(eq={"EmployeeID": "A4A4-F49C"}),
]

In [5]:
features = dataset.features(
    filters=filters,
    transform=transform,
)

In [6]:
features.collect()

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from PerformanceRating.csv,max(RelationshipSatisfaction) from PerformanceRating.csv,mean(RelationshipSatisfaction) from PerformanceRating.csv,median(RelationshipSatisfaction) from PerformanceRating.csv,min(RelationshipSatisfaction) from PerformanceRating.csv,std(RelationshipSatisfaction) from PerformanceRating.csv,sum(RelationshipSatisfaction) from PerformanceRating.csv,var(RelationshipSatisfaction) from PerformanceRating.csv,max(ManagerRating) from PerformanceRating.csv,mean(ManagerRating) from PerformanceRating.csv,median(ManagerRating) from PerformanceRating.csv,min(ManagerRating) from PerformanceRating.csv,std(ManagerRating) from PerformanceRating.csv,sum(ManagerRating) from PerformanceRating.csv,var(ManagerRating) from PerformanceRating.csv,max(EnvironmentSatisfaction) from PerformanceRating.csv,mean(EnvironmentSatisfaction) from PerformanceRating.csv,median(EnvironmentSatisfaction) from PerformanceRating.csv,min(EnvironmentSatisfaction) from PerformanceRating.csv,std(EnvironmentSatisfaction) from PerformanceRating.csv,sum(EnvironmentSatisfaction) from PerformanceRating.csv,var(EnvironmentSatisfaction) from PerformanceRating.csv,max(SelfRating) from PerformanceRating.csv,mean(SelfRating) from PerformanceRating.csv,median(SelfRating) from PerformanceRating.csv,min(SelfRating) from PerformanceRating.csv,std(SelfRating) from PerformanceRating.csv,sum(SelfRating) from PerformanceRating.csv,var(SelfRating) from PerformanceRating.csv,…,min(YearsAtCompany) / median(StockOptionLevel) from Employee.csv,var(Age) + var(StockOptionLevel) from Employee.csv,var(Age) - var(StockOptionLevel) from Employee.csv,var(StockOptionLevel) - var(Age) from Employee.csv,var(Age) * var(StockOptionLevel) from Employee.csv,var(Age) / var(StockOptionLevel) from Employee.csv,var(StockOptionLevel) / var(Age) from Employee.csv,var(Age) + min(StockOptionLevel) from Employee.csv,var(Age) - min(StockOptionLevel) from Employee.csv,min(StockOptionLevel) - var(Age) from Employee.csv,var(Age) * min(StockOptionLevel) from Employee.csv,var(Age) / min(StockOptionLevel) from Employee.csv,min(StockOptionLevel) / var(Age) from Employee.csv,var(Age) + min(YearsAtCompany) from Employee.csv,var(Age) - min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) - var(Age) from Employee.csv,var(Age) * min(YearsAtCompany) from Employee.csv,var(Age) / min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) / var(Age) from Employee.csv,var(StockOptionLevel) + min(StockOptionLevel) from Employee.csv,var(StockOptionLevel) - min(StockOptionLevel) from Employee.csv,min(StockOptionLevel) - var(StockOptionLevel) from Employee.csv,var(StockOptionLevel) * min(StockOptionLevel) from Employee.csv,var(StockOptionLevel) / min(StockOptionLevel) from Employee.csv,min(StockOptionLevel) / var(StockOptionLevel) from Employee.csv,var(StockOptionLevel) + min(YearsAtCompany) from Employee.csv,var(StockOptionLevel) - min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) - var(StockOptionLevel) from Employee.csv,var(StockOptionLevel) * min(YearsAtCompany) from Employee.csv,var(StockOptionLevel) / min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) / var(StockOptionLevel) from Employee.csv,min(StockOptionLevel) + min(YearsAtCompany) from Employee.csv,min(StockOptionLevel) - min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) - min(StockOptionLevel) from Employee.csv,min(StockOptionLevel) * min(YearsAtCompany) from Employee.csv,min(StockOptionLevel) / min(YearsAtCompany) from Employee.csv,min(YearsAtCompany) / min(StockOptionLevel) from Employee.csv
u32,i64,f64,f64,i64,f64,i64,f64,u32,i64,f64,f64,i64,f64,i64,f64,i64,f64,f64,i64,f64,i64,f64,i64,f64,f64,i64,f64,i64,f64,i64,f64,f64,i64,f64,i64,f64,…,f64,